In [1]:
from fastai.text import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
import sentencepiece as spm
import re
import pdb

In [2]:
import fastai, torch
fastai.__version__ , torch.__version__

('1.0.57', '1.0.0')

In [3]:
torch.cuda.set_device(0)

In [4]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)  
    torch.manual_seed(seed_value)  
    random.seed(seed_value)
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)  
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [5]:
random_seed(42, True)

In [6]:
path = Path('./')

In [7]:
!pwd

/data/home/ubuntu/gaurav/in/nlp-for-tamil/classification


In [8]:
df_train = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ta/ta-train.csv', header=None)
df_train.head()

,0,1
0,tamil-cinema,கே.வி.ஆனந்தே ட்விட்டரில் இதை அறிவித்துள்ளார். ...
1,spirituality,"பரிகாரம்: சுக்கிரன், கேதுவுக்குப் பிரீதி, பரிக..."
2,business,8 லட்சம் கார்களை திரும்பப் பெற்றது ஹோண்டா நிறு...
3,business,2003-ம் ஆண்டு அக்டோபர் மாதத்திலிருந்து 2013-ம்...
4,spirituality,தொழில் வியாபாரத்தில் பணத்தேவை ஏற்படலாம். கடன் ...


In [9]:
df_valid = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ta/ta-valid.csv', header=None)
df_valid.head()

,0,1
0,tamil-cinema,"'4ஜி', 'சர்வம் தாளமயம்', 'அடங்காதே' உள்ளிட்ட ப..."
1,spirituality,பாண்டிச்சேரியில் அம்பலத்தாடையார் மடத்து வீதியி...
2,business,விளைபொருட்களை உரிய விலையில் சந்தைப்படுத்த 10 ம...
3,spirituality,"’காஞ்சிபுரத்திலே ஸ்ரீசக்ரம், மாங்காட்டிலே அர்த..."
4,spirituality,ஈசன் புளிய மரத்தருகே தோன்றியதால் புளிய மரம் தல...


In [10]:
df_test = pd.read_csv(path/'../../classification_public_datasets/inltk-headlines/ta/ta-test.csv', header=None)
df_test.head()

,0,1
0,spirituality,மாலவனின் மீது மாளாத அன்பும் பக்தியும் கொண்டு ப...
1,tamil-cinema,"நண்பன் காதலுக்கும், தன் காதலுக்கும் சேர்ந்து ப..."
2,tamil-cinema,ஒரு திரைப்படத்தைப் பார்க்கும் பலருக்கும் பலவித...
3,spirituality,"செவ்வாயின் சித்திரை நட்சத்திரம் 3, 4-ம் பாதம் ..."
4,spirituality,அந்தச் சமயத்தில் தாளக் கருவிகளோடு மணியும் சேர்...


In [11]:
df_train.shape, df_valid.shape, df_test.shape

((5346, 2), (669, 2), (669, 2))

In [12]:
df_train[df_train[0].isnull()].shape, df_valid[df_valid[0].isnull()].shape, df_test[df_test[0].isnull()].shape

((0, 2), (0, 2), (0, 2))

In [13]:
class TamilTokenizer(BaseTokenizer):
    def __init__(self, lang:str):
        self.lang = lang
        self.sp = spm.SentencePieceProcessor()
        self.sp.Load(str('./../../models/tamil/tokenizer/tamil_spm_8k.model'))
        
    def tokenizer(self, t:str) -> List[str]:
        return self.sp.EncodeAsPieces(t)

In [14]:
sp = spm.SentencePieceProcessor()
sp.Load(str('./../../models/tamil/tokenizer/tamil_spm_8k.model'))
itos = [sp.IdToPiece(int(i)) for i in range(8000)]

In [15]:
tamil_vocab = Vocab(itos)

In [16]:
tokenizer = Tokenizer(tok_func=TamilTokenizer, lang='ta')

In [17]:
label_cols = [0]

In [18]:
data_lm = TextLMDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab)

In [19]:
data_lm.show_batch()

idx,text
0,"ன்ஸ் ▁நிறுவனம் ▁தயாரிக்க ிறது . ▁ x x b os ▁பரி கார ம் : ▁சு க்கிர ன் , ▁கே து வுக்கு ப் ▁பி ரீ தி , ▁பரி கார ங்கள் ▁செய்து ▁கொள்ள வும் . ▁ x x b os ▁8 ▁லட்ச ம் ▁கார் களை ▁திரும்ப ப் ▁பெற்றது ▁ஹோ ண்டா ▁நிறுவனம் ▁ x x b os ▁2003 - ம் ▁ஆண்டு ▁அக்டோபர் ▁மாத த்திலிருந்து ▁2013 - ம் ▁ஆண்டு ▁மார்ச் ▁மாதம் ▁வரை ▁ஜெனரல் ▁டை"
1,"ார் ந்த ▁நன் றி யை ▁தெரிவித்த ுக் ▁கொள் கிற ேன் ' ' ▁என்று ▁விஜய் ▁கூறியுள்ளார் . ▁ x x b os ▁ஆதார ் ▁பதிவு ▁இயந்திர ங்களை ▁கொள் முதல் ▁செய்ய ▁வங்கி களுக்கு ▁அனுமதி ▁ x x b os ▁இந்தப் ▁படம் ▁தமிழ் , ▁தெலுங்கு ▁என்று ▁இரண்டு ▁மொழி களிலும் ▁ரி லீ ஸ ாகப் ▁போ கிறது . ▁இனி ▁நீ ங்கள் ▁தொடர்ந்து ▁ தெலுங்க ிலும் ▁கவனம் ▁செலுத்த ு வீ ர் களா ? ▁ x x"
2,ம் ▁காட்ட ாமல் ▁அன்ப ாக ▁பேச ுவது ▁நல்ல து . ▁மகிழ்ச்சி ▁உண்டாகும் . ▁ x x b os ▁ராம் சு ர த் கு ன் வர் ▁எனும் ▁சிறு வன் . . . ▁16 ▁வயது ▁வா லி பன் ▁தன்னை ச் ▁சுற்றியுள்ள ▁எல்லா ▁விஷயங்கள ையும் ▁உள்வாங்க ிக் ▁கொண்ட ான் . ▁தான் ▁செய்யும் ▁எல்லா க் ▁கா ரிய ங்களிலும் ▁முழு து மாக த் ▁தன்னை ▁ஒ ப்படை த்தான் . ▁ x x b os ▁அனு தி
3,x x b os ▁குறைந்த ▁விலை ▁கார் ▁என்ற ▁பிரா ண்டி ல் ▁அறிமுக மான ▁நா னோ ▁இன்று ▁மூ டு விழா ▁காண ும் ▁நிலைக்கு த் ▁தள்ள ப்பட்டு ▁ விட்டது . ▁ x x b os ▁தொலை த் ▁தொடர்பு ▁சேவை ▁அளிக்கும் ▁நிறுவனங்கள் ▁இடை யூ றி ல்லா ▁சேவை ▁அளிக்க ▁ராணுவ த்துக்கு ▁சொந்தமான ▁இடங்களில் ▁செல் போ ன் ▁ட வர்கள் ▁அமைக்க ▁அனுமதிக்க ப்படும் ▁என்று ▁மத்திய ▁தகவல் ▁தொழில்நுட்ப த் ▁துறை ▁அமைச்சர் ▁ரவி சங்க ர் ▁பிரசாத் ▁தெரிவித்தார் .
4,"த்தார் ▁அனைவரும் ▁விழுந்த ு ▁ந ம ஸ் கர ித்தார் கள் . ▁ x x b os ▁பு வன் ▁ஆர் . ந ல்லா ன் ▁இயக்கத்தில் ▁ டபிள்யூ டி எஃப் ▁என் டர் டெ யின் மென் ட் ▁சார்பில் ▁ரோ ஹி த் ▁ர மே ஷ் ▁மற்றும் ▁ஹரி ▁ கிருஷ்ணன் ▁ஆகியோர் ▁இணைந்து ▁தயாரிக்க ும் ▁‘ மோ ’ ▁படத்தின் ▁பு ரொ மோ ஷன் , ▁புத்த ாண்டு ▁கொண்டாட்ட ம் ▁என்று ▁பர பர ப்பாக ▁இருந்த ▁அவ ரிடம் ▁ஒரு"


In [20]:
awd_lstm_config = awd_lstm_lm_config.copy()
awd_lstm_config['n_hid'] = 1150
learn = language_model_learner(data_lm, arch=AWD_LSTM, drop_mult=0.3, config=awd_lstm_config, pretrained=False)

In [21]:
learn.load('../../../models/tamil/lm/wikitalm_8k_447_third')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (5346 items)
x: LMTextList
▁ x x b os ▁கே . வி . ஆ ன ந்தே ▁ ட் விட்ட ரில் ▁இதை ▁அறிவித்த ுள்ளார் . ▁இந்தப் ▁பட த்துக்கு ▁கே வ் மி க் ▁ஆ ரி ▁ஒளிப்பதி வு ▁செய்ய , ▁ஹா ரி ஸ் ▁ஜெய ராஜ் ▁இசை ய மை க்கிறார் . ▁பட்டு க்கோட்டை ▁பிர பா கர் ▁வசன ம் ▁எழுத , ▁கலை ▁இயக்குநராக ▁கிர ண் ▁பணியாற்ற ுகிறார் . ▁இந்தப் ▁பட த்தை ▁ லை கா ▁பு ரொ ட க் ▁ஷ ன்ஸ் ▁நிறுவனம் ▁தயாரிக்க ிறது .,▁ x x b os ▁பரி கார ம் : ▁சு க்கிர ன் , ▁கே து வுக்கு ப் ▁பி ரீ தி , ▁பரி கார ங்கள் ▁செய்து ▁கொள்ள வும் .,▁ x x b os ▁8 ▁லட்ச ம் ▁கார் களை ▁திரும்ப ப் ▁பெற்றது ▁ஹோ ண்டா ▁நிறுவனம்,▁ x x b os ▁2003 - ம் ▁ஆண்டு ▁அக்டோபர் ▁மாத த்திலிருந்து ▁2013 - ம் ▁ஆண்டு ▁மார்ச் ▁மாதம் ▁வரை ▁ஜெனரல் ▁டை ன மி க் ▁கார் ப்ப ரே ஷன் ▁நிறுவனத்தின் ▁நிர்வாக த் ▁துணை த்தலைவர் ▁பொறுப்ப ில் ▁இருந்தவர் .,▁ x x b os ▁தொழில் ▁வியாபார த்தில் ▁பண த்தே வை ▁ஏற்படலாம் . ▁கடன் ▁விவகார ங்களில் ▁கவன மாக ▁செயல்பட ுவது ▁நல்ல து . ▁உத்தி யோ க த்தில் ▁இருப்ப வர்கள் ▁ஓய்வு ▁இல்லாமல் ▁பணியாற்ற ▁வேண்டி ▁இருக்கும் . ▁இயந்

In [22]:
learn.freeze()

In [23]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.135746,3.873413,0.349617,00:04


In [24]:
learn.unfreeze()

In [25]:
learn.fit_one_cycle(5, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.787318,3.718931,0.366327,00:05
1,3.626427,3.568200,0.383865,00:05
2,3.444955,3.501889,0.393208,00:05
3,3.307620,3.482186,0.396333,00:05
4,3.224816,3.479567,0.397034,00:05


In [26]:
learn.predict('▁தனது ▁சொந்த',n_words=10)

'▁தனது ▁சொந்த ▁ஊரில் ▁முனை ப்போ டு ▁ஆர் யு பு க்கு ▁‘ கு'

In [27]:
learn.save_encoder('fine_tuned_enc')

In [28]:
data_clas = TextClasDataBunch.from_df(path=path, train_df=df_train, valid_df=df_valid, test_df=df_test, tokenizer=tokenizer, vocab=tamil_vocab, label_cols=label_cols, bs=64)

In [29]:
data_clas.show_batch()

text,target
"▁ x x b os ▁நடிகர் ▁ச சி குமார் ▁மிகவும் ▁சிறந்த ▁இயக்குநர் ▁மற்றும் ▁நல்ல ▁மனிதர் . ▁அவரது ▁உறவினர் ▁அ சோ க்கு மர் ▁தற்கொலை யை ▁நினைத்த ு ▁நான் ▁மிகவும் ▁மன வே தனை ப்படுகிற ேன் . ▁அசோக ் குமார் ▁தனது ▁மனைவி , ▁பெற்றோர் ▁மற்றும் ▁குழந்தை களை ▁தவ ிக்க விட்டு விட்டு ▁தற்கொலை ▁செய்யும் ▁முடி வை ▁தவிர ் த்த ிருக்க ▁வேண்டும் . ▁ ta m il - c ine ma | ந டி கை",tamil-cinema
▁ x x b os ▁நா னும் ▁கமல ் ஹா சன ும் ▁ஒரு ▁படத்தில் ▁இணைய ும் ▁சூழல் ▁வந்தது . ▁கதை ▁விவாத ம் ▁செய்த ோம் . ▁அது ▁பௌத்த ▁சித்தாந்த ங்களைப் ▁பற்றிய ▁ஒரு ▁படம் . ▁புத்தர் ▁வாழ்ந்த ▁கால த்துக்கு ப் ▁பிறகு ▁நடப்ப து ▁போன்ற ▁கதை . ▁அவரது ▁பல் ▁எப்படி ▁இலங்கை ▁வரை ▁சென்ற து ▁என்பது தான் ▁படத்தின் ▁ஒரு வரி க் ▁கதை . ▁ ta m il - c ine ma | உ ண்மை,tamil-cinema
"▁ x x b os ▁இவ்வி ழா ▁ரொ ம்ப ▁ஈர்க்க ும ளவு க்கு ▁இருந்தது . ▁இந்த ▁விழா வைப் ▁பார்த்த வுடன் ▁2. 0 ▁இசை ▁வெளியீட்டு ▁விழா வை ▁எப்படி ▁ப ண்ண லாம் ▁என்ற ▁ஐ டியா ▁எல்லாம் ▁வந்த ிருக்கிறது . ▁' ரெ மோ ' ▁ரொ ம்ப ▁ஈர்க்க க்கூடிய ▁ஒரு ▁படம் . ▁நடிகர் கள் , ▁தொழில்நுட்ப க் ▁கலைஞர் கள் ▁என ▁அனைத்து ▁வகை யிலும் ▁ரொ ம்ப ▁கவனம் ▁செலுத்த ிய ிருக்கிறார் கள் . ▁b us in",tamil-cinema
"▁ x x b os ▁இந்தியாவில் ▁புதிய ▁வரி ▁முறையான ▁ஜி எஸ் டி ▁அ மல் படுத்த ப்பட்டது ▁மிகப்பெரிய ▁பொருளாதார ▁மாற ுதலை ▁ஏற்படுத்திய ுள்ளது . ▁இதனால் ▁உடனடியாக ▁சில ▁பாதிப்பு கள் ▁இருந்தன . ▁பொருளாதார ▁வளர்ச்சி ▁விகிதம் ▁குறை யும் ▁என்ற ▁எதிர் பார் ப்பு ▁இருந்தது . ▁b us in ess | ந ட ப்பு ▁நிதி யா ண்டி ல் ▁பொருளாதார ▁வளர்ச்சி ▁6 . 5 ▁சதவீத மாக ▁இருக்கும் ▁என ▁இந்திய ▁அரசு ▁மதிப்ப ிட்ட ுள்ளது . ▁ஆனால் ,",business
▁ x x b os <unk> ▁a s ▁ sp ec ul ate d ▁m y ▁n ex t ▁ d ir ect or ial ▁ v en ture ▁w ill ▁ be ▁w ith ▁ <unk> <unk> <unk> n and al f il m s ▁ <unk> ▁ d d 2 . <unk> ▁w ill ▁ al s o ▁ be ▁ pl ay ing ▁the ▁ le ad,tamil-cinema


In [30]:
del awd_lstm_config['tie_weights']
del awd_lstm_config['out_bias']

In [31]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM, drop_mult=0.5, config=awd_lstm_config)

In [32]:
learn.load_encoder('fine_tuned_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (5346 items)
x: TextList
▁ x x b os ▁கே . வி . ஆ ன ந்தே ▁ ட் விட்ட ரில் ▁இதை ▁அறிவித்த ுள்ளார் . ▁இந்தப் ▁பட த்துக்கு ▁கே வ் மி க் ▁ஆ ரி ▁ஒளிப்பதி வு ▁செய்ய , ▁ஹா ரி ஸ் ▁ஜெய ராஜ் ▁இசை ய மை க்கிறார் . ▁பட்டு க்கோட்டை ▁பிர பா கர் ▁வசன ம் ▁எழுத , ▁கலை ▁இயக்குநராக ▁கிர ண் ▁பணியாற்ற ுகிறார் . ▁இந்தப் ▁பட த்தை ▁ லை கா ▁பு ரொ ட க் ▁ஷ ன்ஸ் ▁நிறுவனம் ▁தயாரிக்க ிறது .,▁ x x b os ▁பரி கார ம் : ▁சு க்கிர ன் , ▁கே து வுக்கு ப் ▁பி ரீ தி , ▁பரி கார ங்கள் ▁செய்து ▁கொள்ள வும் .,▁ x x b os ▁8 ▁லட்ச ம் ▁கார் களை ▁திரும்ப ப் ▁பெற்றது ▁ஹோ ண்டா ▁நிறுவனம்,▁ x x b os ▁2003 - ம் ▁ஆண்டு ▁அக்டோபர் ▁மாத த்திலிருந்து ▁2013 - ம் ▁ஆண்டு ▁மார்ச் ▁மாதம் ▁வரை ▁ஜெனரல் ▁டை ன மி க் ▁கார் ப்ப ரே ஷன் ▁நிறுவனத்தின் ▁நிர்வாக த் ▁துணை த்தலைவர் ▁பொறுப்ப ில் ▁இருந்தவர் .,▁ x x b os ▁தொழில் ▁வியாபார த்தில் ▁பண த்தே வை ▁ஏற்படலாம் . ▁கடன் ▁விவகார ங்களில் ▁கவன மாக ▁செயல்பட ுவது ▁நல்ல து . ▁உத்தி யோ க த்தில் ▁இருப்ப வர்கள் ▁ஓய்வு ▁இல்லாமல் ▁பணியாற்ற ▁வேண்டி ▁இருக்கும் . ▁இயந்திர ம

In [33]:
learn.freeze()

In [34]:
learn.loss_func.func

CrossEntropyLoss()

In [35]:
mcc = MatthewsCorreff()

In [36]:
learn.metrics = [mcc, accuracy]

In [37]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.281653,0.168225,0.909438,0.940209,00:05


In [38]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.211382,0.139259,0.932308,0.955157,00:05


In [39]:
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3, callbacks=[callbacks.SaveModelCallback(learn, every='improvement', monitor='accuracy', name='final')])

epoch,train_loss,valid_loss,matthews_correff,accuracy,time
0,0.115121,0.120591,0.940761,0.961136,00:10
1,0.091555,0.119007,0.938388,0.959641,00:12
2,0.070927,0.119227,0.936555,0.958147,00:11
3,0.050470,0.130328,0.929559,0.953662,00:10
4,0.046812,0.121785,0.931623,0.955157,00:11


Better model found at epoch 0 with accuracy value: 0.9611360430717468.


In [40]:
from sklearn.metrics import accuracy_score, matthews_corrcoef
df_dict = {'query': list(df_test[1]), 'actual_label': list(df_test[0]), 'predicted_label': ['']*df_test.shape[0]}
all_nodes = list(set(df_train[0]))
for node in all_nodes:
    df_dict[node] = ['']*df_test.shape[0]
    
i2c = {}
for key, value in learn.data.c2i.items():
    i2c[value] = key
    
df_result = pd.DataFrame(df_dict)
preds = learn.get_preds(ds_type=DatasetType.Test, ordered=True)
for index, row in df_result.iterrows():
    for node in all_nodes:
        row[node] = preds[0][index][learn.data.c2i[node]].item()
    row['predicted_label'] = i2c[np.argmax(preds[0][index]).data.item()]
df_result.head()

,query,actual_label,predicted_label,tamil-cinema,spirituality,business
0,மாலவனின் மீது மாளாத அன்பும் பக்தியும் கொண்டு ப...,spirituality,spirituality,0.000588091,0.998337,0.00107544
1,"நண்பன் காதலுக்கும், தன் காதலுக்கும் சேர்ந்து ப...",tamil-cinema,tamil-cinema,0.992359,0.00406395,0.00357747
2,ஒரு திரைப்படத்தைப் பார்க்கும் பலருக்கும் பலவித...,tamil-cinema,tamil-cinema,0.985132,0.00887207,0.00599544
3,"செவ்வாயின் சித்திரை நட்சத்திரம் 3, 4-ம் பாதம் ...",spirituality,spirituality,7.77528e-05,0.999504,0.000418323
4,அந்தச் சமயத்தில் தாளக் கருவிகளோடு மணியும் சேர்...,spirituality,spirituality,0.0794979,0.888068,0.0324337


In [41]:
accuracy_score(df_result['actual_label'], df_result['predicted_label'])

0.9521674140508222

In [42]:
matthews_corrcoef(df_result['actual_label'], df_result['predicted_label'])

0.9269514834025239

In [43]:
df_result.to_csv('inltk_headlines_ta.csv', index=False)